In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
import os
import tensorflow as tf
from tensorflow import keras

In [ ]:
# Download dataset
dataset_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
data_dir = tf.keras.utils.get_file(
    'flower_photos', origin=dataset_url, cache_dir=".", untar=True)


In [ ]:
data_dir

In [ ]:
import pathlib
data_dir = pathlib.Path(data_dir)
data_dir

In [ ]:
roses_list = list(data_dir.glob('roses/*'))
roses_list[:5]

In [ ]:
#  read flowers from disk into numpy array using openCV

flowers_image_dict = {
    'roses': list(data_dir.glob("roses/*")),
    'daisy': list(data_dir.glob('daisy/*')),
    'dandelion': list(data_dir.glob('dandelion/*')),
    'sunflowers': list(data_dir.glob('sunflowers/*')),
    'tulips': list(data_dir.glob('tulips/*')),
}

flowers_label_dict = {
    'roses': 0,
    'daisy': 1,
    'dandelion': 2,
    'sunflowers': 3,
    'tulips': 4,
}


In [ ]:
# create your X and y
X,y = [],[]

for flower_name, images in flowers_image_dict.items():
    for image in images:
        img = cv2.imread(str(image))
        resized = cv2.resize(img, (180,180))
        X.append(resized)
        y.append(flowers_label_dict[flower_name])

X = np.array(X)
y = np.array(y)




In [ ]:
# train, test, split
from sklearn.model_selection import train_test_split

X_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)


In [ ]:
# scaling

X_train = X_train / 255
x_test = x_test / 255

In [ ]:
num_classes = 5

data_augmentation = keras.Sequential(
  [
    keras.layers.experimental.preprocessing.RandomFlip("horizontal", 
                                                 input_shape=(180, 
                                                              180,
                                                              3)),
    keras.layers.experimental.preprocessing.RandomRotation(0.1),
    keras.layers.experimental.preprocessing.RandomZoom(0.1),
  ]
)

cnn = keras.models.Sequential([
data_augmentation,
  keras.layers.Conv2D(16, 3, padding='same', activation='relu'),
  keras.layers.MaxPooling2D(),
  keras.layers.Conv2D(32, 3, padding='same', activation='relu'),
  keras.layers.MaxPooling2D(),
  keras.layers.Conv2D(64, 3, padding='same', activation='relu'),
  keras.layers.MaxPooling2D(),
  keras.layers.Dropout(0.2),
  keras.layers.Flatten(),
  keras.layers.Dense(128, activation='relu'),
  keras.layers.Dense(num_classes)
])

cnn.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])


In [ ]:
cnn.fit(X_train, y_train, epochs=10)

In [ ]:
cnn.evaluate(x_test, y_test)

In [ ]:
y_predict = cnn.predict(x_test)


In [ ]:
y_answer = [np.argmax(elements) for elements in y_predict]
y_answer[:10]

In [ ]:
y_test[:10]